In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import seaborn as sns
from collections import Counter
from pyinform import transfer_entropy
from pyinform import mutual_info
from statsmodels.tsa.stattools import grangercausalitytests
colors_to_numbers = {'gru': 0, 'pty': 1, 'jfk': 2, 'lax': 3, 'las': 4, 'lis': 5, 'bru': 6, 'hkg': 7, 'icn': 8, 'mex': 9}

In [4]:
def reorg_dataframe_at_a_timestep(data_frame_row_as_list):
    X = data_frame_row_as_list
    q1,q2,q3,q4 = X[:100],X[100:200],X[200:300],X[300:400]
    matrix_Q1, matrix_Q2, matrix_Q3, matrix_Q4 = np.array(q1).reshape(10, 10),np.array(q2).reshape(10, 10),np.array(q3).reshape(10, 10), np.array(q4).reshape(10, 10)
    top_half = np.concatenate((matrix_Q2, matrix_Q1), axis=1)  # Q1 and Q2 side-by-side
    bottom_half = np.concatenate((matrix_Q3, matrix_Q4), axis=1)  # Q3 and Q4 side-by-side
    z = np.concatenate((top_half, bottom_half), axis=0)
    return z

def get_neighborhood_sum(dataframe,row,col):
    z = dataframe
    neighbor_TL = z[row-1][col-1]
    neighbor_T = z[row-1][col]
    neighbor_TR = z[row-1][(col+1)%20]
    neighbor_L = z[row][col-1]
    neighbor_R = z[row][(col+1)%20]
    neighbor_BL = z[(row+1)%20][col-1]
    neighbor_B = z[(row+1)%20][col]
    neighbor_BR = z[(row+1)%20][(col+1)%20]
    return (neighbor_TL + neighbor_T +neighbor_TR + neighbor_L + neighbor_R + neighbor_BL + neighbor_B + neighbor_BR)

def baseline_cell_to_matrix(baseline_cell):
    baseline_cell = baseline_cell-1
    if baseline_cell<=100:
        row = baseline_cell//10%10
        col = baseline_cell%10
        col_offset = 10
        return baseline_cell//10%10, (baseline_cell%10)+10
    elif baseline_cell<=200:
        row = baseline_cell//10%10
        col = baseline_cell%10
        col_offset = 10
        return baseline_cell//10%10, (baseline_cell%10)
    elif baseline_cell<=300:
        row = baseline_cell//10%10
        col = baseline_cell%10
        col_offset = 10
        return (baseline_cell//10%10)+10, (baseline_cell%10)
    else:
        row = baseline_cell//10%10
        col = baseline_cell%10
        col_offset = 10
        return (baseline_cell//10%10)+10, (baseline_cell%10)+10
    return row,col


def get_moore_neighborhood(dataframe,row,col):
    z = dataframe
    neighbor_TL = z[row-1][col-1]
    neighbor_T =  z[row-1][col]
    neighbor_TR = z[row-1][(col+1)%20]
    neighbor_L =  z[row][col-1]
    cell =        z[row][col]
    neighbor_R =  z[row][(col+1)%20]
    neighbor_BL = z[(row+1)%20][col-1]
    neighbor_B =  z[(row+1)%20][col]
    neighbor_BR =  z[(row+1)%20][(col+1)%20]
    neighborhood = [neighbor_TL, neighbor_T, neighbor_TR, neighbor_L, cell,neighbor_R,neighbor_BL,neighbor_B,neighbor_BR]
    return neighborhood


In [23]:
#we are going to look for causal rules between cell 255 and cell cell 55---> we are trying to predict if the state of cell 255 is determined by the behavior of cell 55
# to do this lets first get all the times that cell 255 changes 

num_df = 15
comparator_state = [0]*400
running_state_transitions = []
running_sums = [0]
neighborhoods = []

for baseline_cell in range(115,116): #cells in range 1 --> 401
    print('BaseLine Cell: ',baseline_cell)
    sum = []
    row,col = baseline_cell_to_matrix(baseline_cell)
    print(row,col)
    for df_counter in range(0, num_df):
        df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_'+str(df_counter)+'.csv').replace(colors_to_numbers) #first read the pdf
        #shifted = df['Q4-pos398'] != df['Q4-pos398'].shift(1)
        shifted =                 df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift(1)
        state_after_transition =  df[df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift()].index 
        state_before_transition = state_after_transition.where(state_after_transition == 0, state_after_transition - 1)
        txn_vals =                df.iloc[:,baseline_cell].loc[state_before_transition].tolist()
        txn_vals_after =          df.iloc[:,baseline_cell].loc[state_after_transition].tolist()
        df_cond = df.loc[state_before_transition]
        df_cond = df_cond.drop(df.columns[0], axis=1)
        #print(df_cond)
        for i in range(1,df_cond.shape[0]):
            #print(i)
            Z = reorg_dataframe_at_a_timestep(df_cond.iloc[i,:].tolist())
            sum.append(get_neighborhood_sum(Z,row,col)) 
            neighborhoods.append(get_moore_neighborhood(Z,row,col))


        running_state_transitions.append(txn_vals_after[1:])

print('The running state transition is:                 ',running_state_transitions, '\n The sum of neighbors / 8 at that tranisiton is: ',sum)
for neighborhood in neighborhoods:
    neighborhood = np.reshape(neighborhood, (3,3))
    print(neighborhood,'\n---------------------------------')

#data = pd.DataFrame({'list1': running_state_transitions, 'list2': sum})
#grangercausalitytests(data,5)

BaseLine Cell:  115
1 4
The running state transition is:                  [[1, 3, 0, 1, 3, 0, 1, 4, 2, 0, 2, 0, 8, 7, 1, 3, 7, 1], [1, 0, 3, 6, 4, 0, 2, 1, 0, 2, 1, 0, 1, 0, 6], [0, 3, 1, 4, 1, 0, 1, 0, 4, 8, 1, 0], [8, 3, 0, 4, 0, 9, 3, 0, 2, 1, 9, 3, 0, 1, 6, 0, 8, 0, 3, 0, 9], [0, 5, 1, 2, 3, 0, 5, 4, 5, 0, 1, 6, 2, 3, 2, 0, 4, 2, 9, 2, 0, 3, 1], [3, 2, 9, 2, 1, 6, 0, 4, 7, 1, 2, 1, 6, 2, 0], [1, 0, 2, 0, 1, 8, 0, 1, 0, 8, 9, 1, 0, 2, 8, 0, 8], [7, 1, 0, 1, 0, 9, 3, 4, 2, 0, 2, 0, 1, 2, 4, 7, 0, 1, 0, 2, 5, 1, 0, 5], [0, 1, 9, 1, 8, 0, 1, 5, 2, 6, 0, 4, 1, 2, 0], [1, 3, 0, 1, 0, 2, 5, 0, 6, 5, 1, 0], [5, 0, 2, 4, 0, 1, 3, 9, 0, 1, 0, 8, 0, 3, 0, 3, 1, 2, 0, 5, 0, 8, 0, 1], [9, 2, 0, 1, 2, 6, 1], [0, 2, 6, 1, 0, 3, 0, 1, 9, 0, 1, 9, 6, 0, 5, 1, 4, 1], [0, 1, 2, 3, 7, 1, 0, 1, 6, 9, 3, 1, 0, 8, 0, 4, 0, 1, 2, 1, 2, 5], [1, 2, 0, 2, 1, 0, 3, 0, 2, 9, 4, 9, 2, 1, 2, 4, 1]] 
 The sum of neighbors / 8 at that tranisiton is:  [15, 26, 10, 17, 18, 27, 12, 14, 14, 14, 20, 17, 15, 15, 22, 19,

In [61]:
num_df = 10
comparator_state = [0]*400
running_state_transitions = []
running_sums = [0]

for baseline_cell in range(56,57): #cells in range 1 --> 401
    print('BaseLine Cell: ',baseline_cell)
    sum = []
    row,col = baseline_cell_to_matrix(baseline_cell)
    print(row,col)
    for df_counter in range(0, num_df):
        df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_'+str(df_counter)+'.csv').replace(colors_to_numbers) #first read the pdf
        #shifted = df['Q4-pos398'] != df['Q4-pos398'].shift(1)
        shifted =                 df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift(1)
        state_after_transition =  df[df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift()].index 
        state_before_transition = state_after_transition.where(state_after_transition == 0, state_after_transition - 1)
        txn_vals =                df.iloc[:,baseline_cell].loc[state_before_transition].tolist()
        txn_vals_after =          df.iloc[:,baseline_cell].loc[state_after_transition].tolist()
        
        df_cond = df.loc[state_before_transition]
        df_cond = df_cond.drop(df.columns[0], axis=1)

        

        for i in range(1,df_cond.shape[0]):
            #print(i)
            Z = reorg_dataframe_at_a_timestep(df_cond.iloc[i,:].tolist())
            
            sum.append(get_neighborhood_sum(Z,row,col)) 


        running_state_transitions.extend(txn_vals_after[1:])


print('The running state transition is:                 ',running_state_transitions, '\n The sum of neighbors / 8 at that tranisiton is: ', [x//8+1 for x in sum])
data = pd.DataFrame({'list1': running_state_transitions, 'list2': sum})
grangercausalitytests(data,5)

BaseLine Cell:  56
5 15
The running state transition is:                  [6, 2, 6, 2, 0, 2, 6, 4, 0, 4, 0, 2, 4, 2, 8, 6, 4, 0, 4, 6, 2, 0, 2, 4, 8, 6, 4, 0, 2, 6, 8, 2, 8, 2, 4, 8, 4, 6, 8, 0, 9, 3, 4, 8, 0, 0] 
 The sum of neighbors / 8 at that tranisiton is:  [5, 5, 3, 3, 2, 5, 4, 3, 3, 6, 4, 3, 3, 1, 1, 1, 1, 1, 4, 4, 3, 3, 5, 5, 6, 5, 6, 3, 4, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 3, 3, 2, 2, 2, 5]

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.8579  , p=0.3596  , df_denom=42, df_num=1
ssr based chi2 test:   chi2=0.9191  , p=0.3377  , df=1
likelihood ratio test: chi2=0.9099  , p=0.3401  , df=1
parameter F test:         F=0.8579  , p=0.3596  , df_denom=42, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7004  , p=0.5025  , df_denom=39, df_num=2
ssr based chi2 test:   chi2=1.5805  , p=0.4537  , df=2
likelihood ratio test: chi2=1.5528  , p=0.4601  , df=2
parameter F test:         F=0.7004  , p=0.5025  , df_denom=39, df_num

{1: ({'ssr_ftest': (0.8578721447398712, 0.3596266864973031, 42.0, 1),
   'ssr_chi2test': (0.9191487265070049, 0.33769858819436493, 1),
   'lrtest': (0.9098875747500301, 0.34014425020233546, 1),
   'params_ftest': (0.8578721447398842, 0.3596266864972991, 42.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (0.7004377493664097, 0.5025041991403605, 39.0, 2),
   'ssr_chi2test': (1.5804749216472833, 0.45373703771806917, 2),
   'lrtest': (1.5527516119880147, 0.46007037779547866, 2),
   'params_ftest': (0.7004377493664029, 0.5025041991403627, 39.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (1.2831337281400292, 0.2948942762880159, 36.0, 3),
   'ssr_chi2test': (4.597895859168437, 0.2037226598596636, 3),
   'lrtest': (4.368302916879912, 0.22434250060317226, 3),
   'params_ftest': (1.2831337281400292, 0.2948942762880159, 36.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0.,

In [9]:
num_df = 199
comparator_state = [0]*400
comparator_cell_values_just_before_txn = []
running_state_transitions = []
running_sums = [0]

for baseline_cell in range(246,247): #cells in range 1 --> 401
    print('BaseLine Cell: ',baseline_cell)
    sum = []
    row,col = baseline_cell_to_matrix(baseline_cell)
    print(row,col)
    comp_cell = 55
    for df_counter in range(0, num_df):
        df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_'+str(df_counter)+'.csv').replace(colors_to_numbers) #first read the pdf
        #shifted = df['Q4-pos398'] != df['Q4-pos398'].shift(1)
        shifted =                 df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift(1)
        state_after_transition =  df[df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift()].index 
        state_before_transition = state_after_transition.where(state_after_transition == 0, state_after_transition - 1)
        txn_vals =                df.iloc[:,comp_cell].loc[state_before_transition].tolist()
        txn_vals_after =          df.iloc[:,baseline_cell].loc[state_after_transition].tolist()
        
        comparator_cell_values_just_before_txn.extend(txn_vals[1:])
        running_state_transitions.extend(txn_vals_after[1:])


print('The running state transition is:                              ',running_state_transitions, '\n The value of the comparison cell just before trnasisiton is: ', comparator_cell_values_just_before_txn)


BaseLine Cell:  246
14 5
The running state transition is:                               [5, 6, 7, 8, 9, 5, 6, 4, 5, 6, 7, 8, 9, 5, 6, 7, 8, 9, 0, 5, 6, 7, 8, 9, 4, 5, 6, 7, 8, 9, 4, 3, 2, 1, 0, 5, 6, 7, 8, 9, 4, 5, 6, 7, 8, 9, 4, 5, 6, 7, 8, 9, 4, 5, 6, 7, 8, 7, 8, 9, 4, 5, 6, 7, 8, 9, 4, 3, 2, 5, 6, 7, 8, 9, 4, 3, 2, 1, 0, 5, 6, 7, 8, 9, 1, 0, 5, 6, 7, 8, 9, 4, 3, 5, 6, 7, 8, 9, 2, 1, 0, 5, 6, 7, 8, 9, 4, 3, 2, 1, 5, 6, 2, 1, 5, 6, 7, 8, 9, 0, 5, 6, 7, 8, 9, 4, 3, 2, 1, 0, 5, 6, 7, 8, 9, 4, 2, 1, 0, 4, 3, 2, 1, 0, 5, 6, 7, 8, 9, 5, 4, 3, 2, 1, 0, 5, 6, 7, 8, 9, 7, 4, 3, 2, 5, 6, 7, 8, 9, 4, 5, 5, 6, 7, 8, 9, 4, 3, 2, 1, 0, 5, 4, 3, 2, 1, 5, 6, 7, 8, 9, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 5, 6, 7, 8, 9, 2, 1, 0, 5, 6, 7, 8, 9, 9, 3, 2, 1, 5, 6, 7, 8, 9, 1, 0, 5, 6, 7, 8, 9, 0, 5, 4, 3, 2, 5, 6, 7, 8, 9, 3, 2, 1, 0, 5, 6, 7, 8, 9, 4, 5, 6, 7, 5, 6, 7, 8, 9, 3, 2, 1, 0, 5, 6, 7, 8, 9, 3, 2, 1, 0, 3, 5, 6, 7, 8, 9, 4, 3, 2, 1, 5, 6, 7, 2, 1, 0, 5, 6, 7, 8, 9, 4, 3, 5, 6, 7, 8, 9, 4, 5, 6, 7, 8,

In [ ]:
#we are going to look for causal rules between cell 255 and cell cell 55---> we are trying to predict if the state of cell 255 is determined by the behavior of cell 55
# to do this lets first get all the times that cell 255 changes 

num_df = 50
comparator_state = [0]*400
running_state_transitions = []
running_sums = [0]
neighborhoods = []
neighborhoods2 = []

for baseline_cell in range(42,43): #cells in range 1 --> 401
    print('BaseLine Cell: ',baseline_cell)
    sum = []
    row,col = baseline_cell_to_matrix(baseline_cell)
    print(row,col)
    for df_counter in range(0, 2):#for df_counter in range(0, num_df):
        df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_'+str(df_counter)+'.csv').replace(colors_to_numbers) #first read the pdf
        #shifted = df['Q4-pos398'] != df['Q4-pos398'].shift(1)
        shifted =                 df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift(1)
        state_after_transition =  df[df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift()].index 
        state_before_transition = state_after_transition.where(state_after_transition == 0, state_after_transition - 1)
        txn_vals =                df.iloc[:,baseline_cell].loc[state_before_transition].tolist()
        txn_vals_after =          df.iloc[:,baseline_cell].loc[state_after_transition].tolist()
        
        
        df_cond = df.loc[state_before_transition]
        df_cond = df_cond.drop(df.columns[0], axis=1)
        #print(df_cond)
        for i in range(1,df_cond.shape[0]):
            #print(i)
            Z = reorg_dataframe_at_a_timestep(df_cond.iloc[i,:].tolist())
            sum.append(get_neighborhood_sum(Z,row,col)) 
            neighborhoods.append(get_moore_neighborhood(Z,row,col))

        df_after = df.loc[state_after_transition]
        df_after = df_after.drop(df.columns[0], axis=1)
        for i in range(1,df_after.shape[0]):
            Z2 = reorg_dataframe_at_a_timestep(df_after.iloc[i,:].tolist())
            neighborhoods2.append(get_moore_neighborhood(Z2,row,col))

        running_state_transitions.append(txn_vals_after)
        #running_state_transitions.extend(txn_vals_after[1:])

    print('The running state transition is:                 ',running_state_transitions, '\n The sum of neighbors / 8 at that tranisiton is: ',sum)
    for index,neighborhood in enumerate(neighborhoods):
        neighborhood = np.reshape(neighborhood, (3,3))
        neighborhood2 = np.reshape(neighborhoods2[index],(3,3))
        print(neighborhood[1][1])
        print(neighborhood2[1][1])
        print(neighborhood,'\n-----------')
        print(neighborhood2,'\n-----------------------------------------')

#data = pd.DataFrame({'list1': running_state_transitions, 'list2': sum})
#grangercausalitytests(data,5)

BaseLine Cell:  42
4 11
The running state transition is:                  [[3, 8, 4, 0], [7, 2, 0]] 
 The sum of neighbors / 8 at that tranisiton is:  [42, 25, 33, 37, 25]
3
8
[[4 8 2]
 [9 3 9]
 [0 5 5]] 
-----------
[[4 8 2]
 [9 8 9]
 [0 5 5]] 
-----------------------------------------
8
4
[[6 8 2]
 [4 8 0]
 [0 5 0]] 
-----------
[[6 8 2]
 [4 4 0]
 [0 5 0]] 
-----------------------------------------
4
0
[[6 8 8]
 [6 4 0]
 [0 5 0]] 
-----------
[[6 8 8]
 [6 0 0]
 [0 5 0]] 
-----------------------------------------
7
2
[[0 2 5]
 [8 7 4]
 [9 7 2]] 
-----------
[[0 2 5]
 [8 2 4]
 [9 7 2]] 
-----------------------------------------
2
0
[[0 2 0]
 [0 2 6]
 [9 0 8]] 
-----------
[[0 2 0]
 [0 0 6]
 [9 0 8]] 
-----------------------------------------


In [76]:
#we are going to look for causal rules between cell 255 and cell cell 55---> we are trying to predict if the state of cell 255 is determined by the behavior of cell 55
# to do this lets first get all the times that cell 255 changes 

num_df = 199
comparator_state = [0]*400
running_state_transitions = []
running_sums = [0]
neighborhoods = []
neighborhoods2 = []

for baseline_cell in range(12,13): #cells in range 1 --> 401
    print('BaseLine Cell: ',baseline_cell)
    sum = []
    row,col = baseline_cell_to_matrix(baseline_cell)
    print(row,col)
    for df_counter in range(0, num_df):#for df_counter in range(0, num_df):
        df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_'+str(df_counter)+'.csv').replace(colors_to_numbers) #first read the pdf
        #shifted = df['Q4-pos398'] != df['Q4-pos398'].shift(1)
        shifted =                 df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift(1)
        state_after_transition =  df[df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift()].index 
        state_before_transition = state_after_transition.where(state_after_transition == 0, state_after_transition - 1)
        txn_vals =                df.iloc[:,baseline_cell].loc[state_before_transition].tolist()
        txn_vals_after =          df.iloc[:,baseline_cell].loc[state_after_transition].tolist()
        
        
        df_cond = df.loc[state_before_transition]
        df_cond = df_cond.drop(df.columns[0], axis=1)
        #print(df_cond)
        for i in range(1,df_cond.shape[0]):
            #print(i)
            Z = reorg_dataframe_at_a_timestep(df_cond.iloc[i,:].tolist())
            sum.append(get_neighborhood_sum(Z,row,col)) 
            neighborhoods.append(get_moore_neighborhood(Z,row,col))

        df_after = df.loc[state_after_transition]
        df_after = df_after.drop(df.columns[0], axis=1)
        for i in range(1,df_after.shape[0]):
            Z2 = reorg_dataframe_at_a_timestep(df_after.iloc[i,:].tolist())
            neighborhoods2.append(get_moore_neighborhood(Z2,row,col))

        running_state_transitions.append(txn_vals_after)
        #running_state_transitions.extend(txn_vals_after[1:])

    print('The running state transition is:                 ',running_state_transitions, '\n The sum of neighbors / 8 at that tranisiton is: ',sum)
    for index,neighborhood in enumerate(neighborhoods):
        neighborhood = np.reshape(neighborhood, (3,3))
        neighborhood2 = np.reshape(neighborhoods2[index],(3,3))
        if(neighborhood[1][1]==2 and neighborhood2[1][1]==4):

            print(neighborhood[1][1])
            print(neighborhood, np.sum(neighborhood),'\n-----------')
            print(neighborhood2[1][1])
            print(neighborhood2,np.sum(neighborhood2),'\n-----------------------------------------')
        
        

#data = pd.DataFrame({'list1': running_state_transitions, 'list2': sum})
#grangercausalitytests(data,5)

BaseLine Cell:  12
1 11
The running state transition is:                  [[1, 4, 2, 6, 2, 8, 0], [2, 6, 2, 6, 0], [8, 6, 8, 0], [1, 0], [0], [3, 6, 0], [0], [8, 0], [6, 2, 0], [4, 8, 0], [7, 4, 0], [0], [1, 3, 0], [9, 3, 6, 0], [5, 0], [2, 8, 0], [3, 0], [2, 4, 6, 2, 0], [9, 8, 0], [5, 0], [8, 0], [5, 0], [5, 0], [9, 4, 6, 8, 2, 0], [4, 0], [6, 0], [8, 0], [4, 0], [4, 0], [0], [6, 2, 6, 8, 4, 2, 4, 0], [2, 6, 0], [5, 0], [2, 0], [6, 4, 2, 0], [2, 6, 0], [9, 6, 0], [2, 0], [9, 1, 0], [9, 1, 0], [0], [1, 6, 0], [2, 6, 0], [4, 0], [3, 0], [5, 0], [2, 4, 0], [4, 2, 0], [0], [6, 2, 0], [9, 8, 6, 8, 6, 0], [2, 6, 8, 6, 4, 0], [3, 2, 0], [4, 6, 4, 0], [2, 0], [5, 0], [2, 4, 6, 0], [7, 5, 0], [7, 5, 0], [1, 7, 8, 6, 0], [7, 1, 9, 8, 2, 8, 0], [5, 0], [7, 0], [7, 5, 0], [5, 0], [9, 7, 3, 7, 4, 0], [6, 8, 0], [2, 8, 6, 4, 0], [6, 4, 0], [6, 0], [2, 8, 0], [7, 0], [9, 3, 0], [9, 0], [5, 0], [2, 0], [4, 6, 2, 6, 0], [4, 0], [5, 0], [4, 2, 8, 0], [0], [3, 0], [7, 2, 6, 8, 4, 6, 0], [6, 2, 6, 0], [